In [2]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import evaluate, print_perf
from surprise import accuracy
from sklearn import cross_validation as cv
from nltk.stem.snowball import SnowballStemmer
from pymongo import MongoClient
%matplotlib inline

/Users/jose/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
client = MongoClient()
db = client['Taller2']
reviews = pd.DataFrame(list(db.review.find()))

In [4]:
reviews

,_id,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,5abc9df63da56a943e90d479,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,5abc9df63da56a943e90d47a,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,5abc9df63da56a943e90d47b,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,5abc9df63da56a943e90d47c,s2I_Ni76bjJNK9yG60iD-Q,0,2016-05-28,0,L_9BTb55X0GDtThi6GlZ6w,4,Had their chocolate almond croissant and it wa...,0,bv2nCi5Qv5vroFiqKGopiw
4,5abc9df63da56a943e90d47d,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw
5,5abc9df63da56a943e90d47e,8QWPlVQ6D-OExqXoaD2Z1g,0,2014-09-24,0,HRPm3vEZ_F-33TYVT7Pebw,5,Cycle Pub Las Vegas was a blast! Got a groupon...,1,_4iMDXbXZ1p1ONG297YEAQ
6,5abc9df63da56a943e90d47f,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
7,5abc9df63da56a943e90d480,9_CGhHMz8698M9-PkVf0CQ,2,2012-05-11,0,ymAUG8DZfQcFTBSOiaNN4w,4,Who would have guess that you would be able to...,0,u0LXt3Uea_GidxRW1xcsfg
8,5abc9df63da56a943e90d481,5r6-G9C4YLbC7Ziz57l3rQ,0,2013-02-09,0,w41ZS9shepfO3uEyhXEWuQ,3,"Not bad!! Love that there is a gluten-free, ve...",1,u0LXt3Uea_GidxRW1xcsfg
9,5abc9df63da56a943e90d482,gkCorLgPyQLsptTHalL61g,0,2015-10-27,0,8UIishPUD92hXtScSga_gw,4,Always drove past this coffee house and wonder...,1,u0LXt3Uea_GidxRW1xcsfg


In [5]:
df = pd.read_csv( '/Users/jose/Documents/Maestria/Sistemas de recomendación/Taller 2/toronto/toronto_reviews.csv')

In [6]:
df = df.rename(columns={'id': 'review_id'})

In [7]:
dfReviews = pd.merge(df, reviews, on='review_id' )

In [8]:
dfReviews['words'] = dfReviews.text.str.strip().str.split('[\W_]+')

In [9]:
dfReviews.head()

,review_id,stars_x,user_id_x,business_id_x,useful_x,cool_x,funny_x,_id,business_id_y,cool_y,date,funny_y,stars_y,text,useful_y,user_id_y,words
0,---0hl58W-sjVTKi5LghGw,4,Y6qylbHq8QJmaCRSlKdIog,Ae4ABFarGMaI5lk1i98A0w,0,0,0,5abc9df83da56a943e91a345,Ae4ABFarGMaI5lk1i98A0w,0,2016-07-24,0,4,Ate the momos during the momo crawl.. Was the ...,0,Y6qylbHq8QJmaCRSlKdIog,"[Ate, the, momos, during, the, momo, crawl, Wa..."
1,---OVMmOizYt30bx3-D0gA,5,WH0D3U5UAtNy0X5U5ZeJnw,ixGb4NLfgD90wZHlw94rLw,6,5,1,5abc9e9d3da56a943ecc8c72,ixGb4NLfgD90wZHlw94rLw,5,2015-08-12,1,5,Went in yesterday to place an order to take ho...,6,WH0D3U5UAtNy0X5U5ZeJnw,"[Went, in, yesterday, to, place, an, order, to..."
2,--0pfY3vQilgl20btE0fVQ,4,wbYWgTS-vzgfVYV_C1qhig,5BPv_vAnRp5EKdmacqvrLw,5,3,2,5abc9ed83da56a943ee11f43,5BPv_vAnRp5EKdmacqvrLw,3,2013-05-19,2,4,An absolute favourite of mine....I don't under...,5,wbYWgTS-vzgfVYV_C1qhig,"[An, absolute, favourite, of, mine, I, don, t,..."
3,--0y7xOZPpiGD5d6vgfafg,1,EUdDBB1fHLq86T0aI9VLtg,vh1tPEaPioD78QmoqnWXpw,1,0,1,5abc9e1c3da56a943ea0c3e9,vh1tPEaPioD78QmoqnWXpw,0,2017-03-12,1,1,Very unimpressed by their pad Thai.. one of th...,1,EUdDBB1fHLq86T0aI9VLtg,"[Very, unimpressed, by, their, pad, Thai, one,..."
4,--1sfdWmiYGbTxpGTy-3ZQ,1,hZgREoZPC30EEilayde2Ug,qAg8GkN9rforFWmGxBqVqA,2,0,0,5abc9e343da56a943eaa8979,qAg8GkN9rforFWmGxBqVqA,0,2016-06-20,0,1,Arrived at 1:40pm on a Sunday just to get flat...,2,hZgREoZPC30EEilayde2Ug,"[Arrived, at, 1, 40pm, on, a, Sunday, just, to..."


In [10]:
rows = list()
for row in dfReviews[['review_id', 'words']].iterrows():
    r = row[1]
    for word in r.words:
        rows.append((r.review_id, word))

words = pd.DataFrame(rows, columns=['review_id', 'word'])

In [11]:
words.describe()

,review_id,word
count,55601337,55601337
unique,431047,214416
top,4u2rngNMwkfac-5qf7Q0zA,the
freq,1026,2304942


In [12]:
words = words[words.word.str.len() > 0]

In [13]:
words.describe()

,review_id,word
count,55197927,55197927
unique,431046,214415
top,4u2rngNMwkfac-5qf7Q0zA,the
freq,1025,2304942


In [14]:
words['word'] = words.word.str.lower()

In [ ]:
counts = words.groupby('review_id')\
    .word.value_counts()\
    .to_frame()\
    .rename(columns={'word':'n_w'})
counts.head()

n_w
review_id              word     
---0hl58W-sjVTKi5LghGw the     6
                       was     2
                       and     1
                       at      1
                       ate     1

In [ ]:
def pretty_plot_top_n(series, top_n=5, index_level=0):
    r = series\
    .groupby(level=index_level)\
    .nlargest(top_n)\
    .reset_index(level=index_level, drop=True)
    r.plot.bar()
    return r.to_frame()


pretty_plot_top_n(counts['n_w'])